# setting

In [1]:
# Load library
import mne, os, pickle, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Qt5Agg')

# load data

In [2]:
# define data directory
subj = 'sub-004'
data_type = 'EEG'
data_dir  = os.path.realpath('../dataset/%s/%s/%s')%(data_type, subj, data_type.lower())

# load eeg data
sess  = 'sess-oddball03'
fname = os.path.join(data_dir, '%s_%s-raw.fif') %(subj, sess)
raw   = mne.io.read_raw(fname, preload=True)

# find events
events = mne.find_events(raw, min_duration=0.002)


Opening raw data file /Users/cl5564/Library/CloudStorage/Dropbox/NYUC&P/Class/Year5-1_2024Fall/MEEG_methods/Lab/dataset/EEG/sub-004/eeg/sub-004_sess-oddball03-raw.fif...
    Range : 1945600 ... 2339840 =   3800.000 ...  4570.000 secs
Ready.
Reading 0 ... 394240  =      0.000 ...   770.000 secs...
362 events found
Event IDs: [  2  64 128]


# re-reference data

In [3]:
# select 256 channels + 4 EOG channels + 1 stimulus channel
pick_chans = raw.ch_names[0:260]     # select 256 channels + 4 EOG channels
pick_chans.append(raw.ch_names[-1])  # append 1 stimulus channel
data_subset = raw.copy()
data_subset.pick(pick_chans)

# re-reference to average channel
# ref_channels='average' --> using the average reference from 256 channels 
data_reref, avg_ref = mne.set_eeg_reference(data_subset, ref_channels='average', copy=True)

# re-reference EOG channels by subtracting "avg_ref"
data_reref._data[256:260,:] -= avg_ref

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


# identify bad channels

In [4]:
data_reref.plot(n_channels=50, duration=30, color='steelblue',
                events=events, event_color='orange', 
                highpass=0.5, lowpass=None, scalings=dict(eeg=2e-5)
               )

Setting up high-pass filter at 0.5 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 8 (effective, after forward-backward)
- Cutoff at 0.50 Hz: -6.02 dB

Using matplotlib as 2D backend.


<MNEBrowseFigure size 2992x2604 with 4 Axes>

Channels marked as bad:
none


In [5]:
# check channels maked as bad
data_reref.info['bads']

['A26',
 'B7',
 'B8',
 'B9',
 'B19',
 'C9',
 'C30',
 'D15',
 'E5',
 'F2',
 'F22',
 'G25',
 'G29']

# interpolate bad channels
Method to use for each channel type.
- "meg" channels support "MNE" (default) and "nan"
- "eeg" channels support "spline" (default), "MNE" and "nan"
- "fnirs" channels support "nearest" (default) and "nan"
- "ecog" channels support "spline" (default) and "nan"
- "seeg" channels support "spline" (default) and "nan"

In [6]:
# original raw data
print(data_subset.info['bads'])

# update bad channels info to the orginal raw data 
data_subset.info['bads'] = data_reref.info['bads']
print(data_subset.info['bads'])

# interpolate bad channels
data_interp = data_subset.copy()
data_interp.interpolate_bads(reset_bads=True, method=dict(eeg='spline'))

[]
['A26', 'B7', 'B8', 'B9', 'B19', 'C9', 'C30', 'D15', 'E5', 'F2', 'F22', 'G25', 'G29']
Interpolating bad channels
    Automatic origin fit: head of radius 95.0 mm
Computing interpolation matrix from 243 sensor positions
Interpolating 13 sensors


<Raw | sub-004_sess-oddball03-raw.fif, 261 x 394241 (770.0 s), ~785.4 MB, data loaded>

# re-reference data again

In [7]:
# re-reference to average channel
# ref_channels='average' --> using the average reference from 256 channels 
data_reref, avg_ref = mne.set_eeg_reference(data_interp, ref_channels='average', copy=True)

# re-reference EOG channels by subtracting "avg_ref"
data_reref._data[256:260,:] -= avg_ref

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


# annotating movement artifacts as bad spans
<div class="alert alert-block alert-info">
    <b>MNE tutorial – Rejecting bad data spans and breaks</b><br>
    <a href="https://mne.tools/stable/auto_tutorials/preprocessing/20_rejecting_bad_data.html">https://mne.tools/stable/auto_tutorials/preprocessing/20_rejecting_bad_data.html</a>
</div>

In [8]:
data_reref.plot(n_channels=50, duration=30, color='steelblue',
                events=events, event_color='orange', 
                highpass=0.5, lowpass=None, scalings=dict(eeg=2e-5)
               )

Setting up high-pass filter at 0.5 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 8 (effective, after forward-backward)
- Cutoff at 0.50 Hz: -6.02 dB

Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [9]:
# # save annotation log file
# annot_fname = os.path.join(data_dir, '%s_%s_annot.csv') %(subj, sess)
# data_reref.annotations.save(annot_fname, overwrite=True)

# print('save annotations')
# print(data_reref.annotations)

# read annotation log file
annot_fname = os.path.join(data_dir, '%s_%s_annot.csv') %(subj, sess)

if os.path.exists(annot_fname):
    annot_badspan = mne.read_annotations(annot_fname)
    data_reref.set_annotations(annot_badspan, emit_warning=False)
    print('read annotations')
    print(data_reref.annotations)
else:
    print('no annotation file!')

read annotations
<Annotations | 10 segments: bad (10)>


## filtering data before ICA

In [10]:
data_filt = data_reref.copy()
data_filt.filter(l_freq=1, h_freq=None, method='fir')

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 1691 samples (3.303 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 256 out of 256 | elapsed:    3.4s finished


<Raw | sub-004_sess-oddball03-raw.fif, 261 x 394241 (770.0 s), ~785.4 MB, data loaded>

## Indepent Component Analysis
<div class="alert alert-block alert-info">
    <b>UCSD SCCN – EEG component labeling</b><br>
    <a href="https://labeling.ucsd.edu/tutorial/labels">https://labeling.ucsd.edu/tutorial/labels</a>
</div>

In [11]:
rnk = np.linalg.matrix_rank(data_filt.get_data())
print(rnk)

247


In [12]:
ica = mne.preprocessing.ICA(n_components=rnk, method='fastica', random_state=1119)
ica.fit(data_filt, reject_by_annotation=True)

Fitting ICA to data using 256 channels (please be patient, this may take a while)
Omitting 66266 of 394241 (16.81%) samples, retaining 327975 (83.19%) samples.
Selecting by number: 247 components


/Users/marcolai/opt/anaconda3/envs/mne_v103/lib/python3.8/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/var/folders/8s/jc0nfbq14wj582qx_r91xgjh0000gn/T/ipykernel_3440/1143468493.py:2: RuntimeWarning: Using n_components=247 (resulting in n_components_=247) may lead to an unstable mixing matrix estimation because the ratio between the largest (1.2e+02) and smallest (2.9e-22) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 242
  ica.fit(data_filt, reject_by_annotation=True)


Fitting ICA took 1775.1s.


Method,fastica
Fit,1000 iterations on raw data (327975 samples)
ICA components,247
Available PCA components,256
Channel types,eeg
ICA components marked for exclusion,—


In [13]:
# save ica file
ica_fname = os.path.join(data_dir, '%s_%s-ica.fif') %(subj, sess)
ica.save(ica_fname, overwrite=True)

# # read ica file
# ica_fname = os.path.join(data_dir, '%s-ica.fif') %(subj)
# ica = mne.preprocessing.read_ica(ica_fname)

Writing ICA solution to /Users/marcolai/Dropbox/NYUC&P/Class/Year5-1_2024Fall/MEEG_methods/Lab/dataset/EEG/sub-004/eeg/sub-004_sess-oddball03-ica.fif...


Method,fastica
Fit,1000 iterations on raw data (327975 samples)
ICA components,247
Available PCA components,256
Channel types,eeg
ICA components marked for exclusion,—


In [14]:
# plot ICA source waveform
ica.plot_sources(data_filt)

# plot the first 40 ICA components
ica.plot_components(picks=np.arange(0,40), nrows=8, ncols=5)


Creating RawArray with float64 data, n_channels=251, n_times=394241
    Range : 1945600 ... 2339840 =   3800.000 ...  4570.000 secs
Ready.
Using pyopengl with version 3.1.6


<MNEFigure size 781x967 with 40 Axes>

In [15]:
# 'sub-004_sess-oddball01': ICA000, ICA001, ICA002 
# 'sub-004_sess-oddball02': ICA000, ICA002, ICA003
# 'sub-004_sess-oddball03': ICA000, ICA001, ICA002

# print marked ICA components
print(ica.exclude)

[0, 1, 2]


In [16]:
# apply ICA to the re-reference data (without filtering)
data_ica = ica.apply(data_reref, exclude=ica.exclude)

Applying ICA to Raw instance
    Transforming to ICA space (247 components)
    Zeroing out 3 ICA components
    Projecting back using 256 PCA components


In [17]:
# save raw file after removing ica components
ica_fname = os.path.join(data_dir, '%s_%s_ICA-raw.fif') %(subj, sess)
data_ica.save(ica_fname, overwrite=True)

Writing /Users/marcolai/Dropbox/NYUC&P/Class/Year5-1_2024Fall/MEEG_methods/Lab/dataset/EEG/sub-004/eeg/sub-004_sess-oddball03_ICA-raw.fif
Closing /Users/marcolai/Dropbox/NYUC&P/Class/Year5-1_2024Fall/MEEG_methods/Lab/dataset/EEG/sub-004/eeg/sub-004_sess-oddball03_ICA-raw.fif
[done]
